# Retrieval


## Import


In [ ]:
from dotenv import load_dotenv
from tqdm import tqdm

from rag_modules import combined_key_query_builder, get_retriever, read_csv_for_rag_query, set_columns_from_config
from utils import format_docs

## Setting


In [ ]:
# 환경 변수 세팅
load_dotenv()

True

In [ ]:
# valid 내 100개만 실험용 valid로 활용
df = read_csv_for_rag_query("data/valid_with_summarization.csv")

test_valid = df[:100]
test_valid.head()

,id,paragraph,question,choices,answer,question_plus,summarization,choices_text,full_question
0,generation-for-nlp-2089,프랑수아 올랑드 프랑스 대통령은 132명의 목숨을 앗아간 ‘11·13 파리 연쇄테러...,프랑수아 올랑드 대통령이 전쟁을 선포한 테러 단체의 이름은 무엇인가?,"[알카에다, 이슬람국가(IS), 탈레반, 하마스, 보코하람]",2,,프랑수아 올랑드 프랑스 대통령은 ‘11·13 파리 연쇄테러’에 맞서 테러 배후인 이...,알카에다 이슬람국가(IS) 탈레반 하마스 보코하람,프랑수아 올랑드 대통령이 전쟁을 선포한 테러 단체의 이름은 무엇인가?
1,generation-for-nlp-2001,강원 원주시가 봉화산2지구에서 공동주택용지를 경쟁입찰 방식으로 공급하려다 돌연 수의...,원주시가 봉화산2지구에서 공동주택용지를 공급하기 위해 처음에 어떤 방식으로 계약을 ...,"[경쟁입찰 방식, 수의계약 방식, 직접 계약 방식, 공모 방식, 임의 계약 방식]",1,,강원 원주시가 봉화산2지구에서 공동주택용지를 경쟁입찰 방식으로 공급하려다 돌연 수의...,경쟁입찰 방식 수의계약 방식 직접 계약 방식 공모 방식 임의 계약 방식,원주시가 봉화산2지구에서 공동주택용지를 공급하기 위해 처음에 어떤 방식으로 계약을 ...
2,generation-for-nlp-2002,한동안 공급이 뜸했던 경기 파주·포천·양주·의정부 등 수도권 북부지역에서 아파트 분...,수도권 북부지역에서 아파트 분양이 증가하고 있는 주된 이유는 무엇인가?,"[전세난과 저금리로 인한 실수요 증가, 부동산 가격 상승으로 인한 투자 수요 증가,...",1,,저금리와 전세난 속에 부동산 경기가 회복세를 보이면서 경기 파주·포천·양주·의정부 ...,전세난과 저금리로 인한 실수요 증가 부동산 가격 상승으로 인한 투자 수요 증가 서울...,수도권 북부지역에서 아파트 분양이 증가하고 있는 주된 이유는 무엇인가?
3,generation-for-nlp-2003,지난 3월 은행 예금회전율이 최근 3년 사이 가장 높은 수준으로 상승했다. 전문가들...,3월 은행 예금회전율이 최근 3년 사이 가장 높은 수준으로 상승한 이유는 무엇인가?,"[은행에 머물던 돈이 주식과 부동산으로 이동하고 있기 때문, 소비자들이 예금을 늘리...",1,,은행에 묶여 있던 돈이 주식 부동산 등 자산시장으로 이동하고 있고 일부 소비 용도로...,은행에 머물던 돈이 주식과 부동산으로 이동하고 있기 때문 소비자들이 예금을 늘리고 ...,3월 은행 예금회전율이 최근 3년 사이 가장 높은 수준으로 상승한 이유는 무엇인가?
4,generation-for-nlp-2835,지난해 정부가 발의한 ‘분양가상한제 탄력 적용(사실상 폐지) 법안’이 이달에도 국회...,분양가 상한제가 처음 도입된 정부는 어느 정부인가?,"[노무현 정부, 이명박 정부, 박근혜 정부, 문재인 정부, 윤석열 정부]",1,,지난해 정부가 발의한 ‘분양가상한제 탄력 적용(사실상 폐지) 법안’이 이달에도 국회...,노무현 정부 이명박 정부 박근혜 정부 문재인 정부 윤석열 정부,분양가 상한제가 처음 도입된 정부는 어느 정부인가?


In [ ]:
# Pinecone retriever 생성
retriever = get_retriever(embedding_model="BAAI/bge-m3", k=5)

Connected to Pinecone index: gen-nlp-bge-m3


## Retrieve


In [ ]:
# config 내 query builder type을 통해 query로 사용할 columns setting
columns = set_columns_from_config(3)
query_builder = combined_key_query_builder(columns)
print(f"사용되는 column list: {columns}")

# retrieve 결과 저장용 리스트
retrieve_results = []

# test_valid 내 행들에 대해 retreive 진행
for _, row in tqdm(test_valid.iterrows(), total=len(test_valid), desc="Processing rows"):

    question_plus_string = ("\n\n<보기>:\n" + row["question_plus"]) if row["question_plus"] else ""
    question = row["question"] + question_plus_string
    choices_string = "\n".join([f"{idx + 1} - {choice}" for idx, choice in enumerate(row["choices"])])

    query = query_builder.build(row)

    retrieved_docs = retriever.invoke(query)

    support = format_docs(retrieved_docs)

    retrieve_results.append(support)

사용되는 column list: ['paragraph', 'full_question', 'choices_text']


Processing rows: 100%|██████████| 100/100 [03:11<00:00,  1.92s/it]


In [ ]:
# 실행 결과 df의 support 열에 저장
test_valid["support"] = retrieve_results
test_valid.head()

C:\Users\yce91\AppData\Local\Temp\ipykernel_26392\1288180729.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_valid["support"] = retrieve_results


,id,paragraph,question,choices,answer,question_plus,summarization,choices_text,full_question,support
0,generation-for-nlp-2089,프랑수아 올랑드 프랑스 대통령은 132명의 목숨을 앗아간 ‘11·13 파리 연쇄테러...,프랑수아 올랑드 대통령이 전쟁을 선포한 테러 단체의 이름은 무엇인가?,"[알카에다, 이슬람국가(IS), 탈레반, 하마스, 보코하람]",2,,프랑수아 올랑드 프랑스 대통령은 ‘11·13 파리 연쇄테러’에 맞서 테러 배후인 이...,알카에다 이슬람국가(IS) 탈레반 하마스 보코하람,프랑수아 올랑드 대통령이 전쟁을 선포한 테러 단체의 이름은 무엇인가?,시리아 내전 개입\n2015년 9월 블라디미르 푸틴 러시아 대통령은 그달 30일에 ...
1,generation-for-nlp-2001,강원 원주시가 봉화산2지구에서 공동주택용지를 경쟁입찰 방식으로 공급하려다 돌연 수의...,원주시가 봉화산2지구에서 공동주택용지를 공급하기 위해 처음에 어떤 방식으로 계약을 ...,"[경쟁입찰 방식, 수의계약 방식, 직접 계약 방식, 공모 방식, 임의 계약 방식]",1,,강원 원주시가 봉화산2지구에서 공동주택용지를 경쟁입찰 방식으로 공급하려다 돌연 수의...,경쟁입찰 방식 수의계약 방식 직접 계약 방식 공모 방식 임의 계약 방식,원주시가 봉화산2지구에서 공동주택용지를 공급하기 위해 처음에 어떤 방식으로 계약을 ...,경매\n입찰\n입찰은 계약체결의 특수한 방법이며 매매·도급(都給) 등의 계약체결에 ...
2,generation-for-nlp-2002,한동안 공급이 뜸했던 경기 파주·포천·양주·의정부 등 수도권 북부지역에서 아파트 분...,수도권 북부지역에서 아파트 분양이 증가하고 있는 주된 이유는 무엇인가?,"[전세난과 저금리로 인한 실수요 증가, 부동산 가격 상승으로 인한 투자 수요 증가,...",1,,저금리와 전세난 속에 부동산 경기가 회복세를 보이면서 경기 파주·포천·양주·의정부 ...,전세난과 저금리로 인한 실수요 증가 부동산 가격 상승으로 인한 투자 수요 증가 서울...,수도권 북부지역에서 아파트 분양이 증가하고 있는 주된 이유는 무엇인가?,2014년 청약시장 과열\n위례신도시는 박근혜 정부의 핵심적인 경제 정책인 초이노믹...
3,generation-for-nlp-2003,지난 3월 은행 예금회전율이 최근 3년 사이 가장 높은 수준으로 상승했다. 전문가들...,3월 은행 예금회전율이 최근 3년 사이 가장 높은 수준으로 상승한 이유는 무엇인가?,"[은행에 머물던 돈이 주식과 부동산으로 이동하고 있기 때문, 소비자들이 예금을 늘리...",1,,은행에 묶여 있던 돈이 주식 부동산 등 자산시장으로 이동하고 있고 일부 소비 용도로...,은행에 머물던 돈이 주식과 부동산으로 이동하고 있기 때문 소비자들이 예금을 늘리고 ...,3월 은행 예금회전율이 최근 3년 사이 가장 높은 수준으로 상승한 이유는 무엇인가?,감안하여 자율적으로 결정하도록 하고 있다.\n 자금의 수요와 공급에 의해 결정된...
4,generation-for-nlp-2835,지난해 정부가 발의한 ‘분양가상한제 탄력 적용(사실상 폐지) 법안’이 이달에도 국회...,분양가 상한제가 처음 도입된 정부는 어느 정부인가?,"[노무현 정부, 이명박 정부, 박근혜 정부, 문재인 정부, 윤석열 정부]",1,,지난해 정부가 발의한 ‘분양가상한제 탄력 적용(사실상 폐지) 법안’이 이달에도 국회...,노무현 정부 이명박 정부 박근혜 정부 문재인 정부 윤석열 정부,분양가 상한제가 처음 도입된 정부는 어느 정부인가?,2014년 청약시장 과열\n위례신도시는 박근혜 정부의 핵심적인 경제 정책인 초이노믹...


# LLM as judge


## Import


In [ ]:
import json
import re

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

## Setting


In [ ]:
# Rubric을 활용한 평가 기준 수립

template = """

당신은 유능한 AI 평가위원입니다.

당신이 할 일은 주어진 문제 데이터(# Question Data)를 푸는데 지원 문서(# Support)가 얼마나 도움이 되는지 평가하는 것입니다.

문제 데이터(# Question Data)는 지문, 문제, 선택지, 정답으로 구성되어 있습니다.

평가 기준(# Evaluation Criteria)은 다음의 3가지 입니다. 


# Evaluation Criteria

1. Support 문서에 문제 풀이에 **필요한 정보**가 전부 포함되었는가?(1점)

정의: 문제를 해결하기 위해 필수적으로 알아야 하는 정보가 문서에 서술되어 있는 경우 점수를 부여합니다. 

점수를 받는 경우 Support를 활용하여 문제의 정답을 맞출 수 있습니다.


2. Support 문서에 문제 풀이에 **필요한 정보**가 일부 포함되었는가?(1점)

정의: 문제를 해결하기 위해 필수적으로 알아야 하는 정보가 문서에 서술되어 있는 경우 점수를 부여합니다. 

점수를 받는 경우 Support를 활용하여 문제의 정답을 맞출 순 없지만, Support의 존재가 문제 풀이에 도움이 됩니다.


3. 문제 풀이에 **필요한 정보**가 Support 문서에서 핵심적으로 다루는 내용인가? (1점)

정의:  문제 풀이에 **필요한 정보**가 Support 문서의 주제나 목적과 밀접하게 관련이 있는 경우 점수를 부여합니다.


평가는 FORMAT에 맞춰 작성하세요.


# FORMAT: 

총점:(예시:2)

각 기준 별 평가 점수:(예시: 1, 1, 0)



# Question Data:
## 문제:

{question}

## 선택지:

{choices}

## 정답:

{answer}


# Support:

{support}

"""

In [ ]:
prompt = PromptTemplate(template=template)

model = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4o",  # 모델명
).bind(logprobs=True)

chain = prompt | model

In [ ]:
responses = []

for _, row in tqdm(test_valid.iterrows(), total=len(test_valid), desc="Processing rows"):
    question_plus = "<보기>" + row["question_plus"] if row["question_plus"] else ""
    question_string = "\n".join([row["question"], question_plus])
    choices_string = "\n".join([f"{idx + 1} - {choice}" for idx, choice in enumerate(row["choices"])])
    input_dict = {
        "paragraph": row["paragraph"],
        "question": question_string,
        "choices": choices_string,
        "answer": row["answer"],
        "support": row["support"],
    }

    # chain 실행
    response = chain.invoke(input_dict)

    # 리스트에 결과 추가
    responses.append(response)

Processing rows: 100%|██████████| 100/100 [06:30<00:00,  3.91s/it]


In [ ]:
# langchacin 결과를 json 파일로 저장
with open("data/langchain_response3.json", "w", encoding="utf-8") as file:

    for response in responses:

        json_line = json.dumps(response.content, ensure_ascii=False)

        file.write(json_line + "\n")

In [ ]:
total_scores = []
total_score = 0

for response in responses:

    input_string = response.content

    # 정규표현식으로 langchain response 내 앞의 숫자 4개만 추출 (총점, 평가항목1, 평가항목2, 평가항목2)
    scores = re.findall(r"\d+", input_string)
    scores = list(map(int, responses))[:4]

    total_score = total_score + scores[0]

    total_scores.append(scores)

In [ ]:
sums = [0, 0, 0, 0]

# 100개의 내부 리스트를 순회하며 합계를 계산
for i in range(len(total_scores)):
    for j in range(4):
        sums[j] += total_scores[i][j]

sums

[18, 5, 12, 1]